Create a local file 

In [ ]:
%%file inflation.txt
Downtown 2.1
Hilltop 4.5

In [ ]:
!cat inflation.txt

Copy the file to the distributed file system HDFS

In [ ]:
!hdfs dfs -copyFromLocal inflation.txt /data/

Verify that the file is replicated 3 times

## Load house prices

In [ ]:
text_RDD = sc.textFile("/data/houses.txt")

In [ ]:
def mapper_parse_lines(line):
    """Parse line into (neighborhoood, price) pair"""
    words = line.split()
    return (words[1], float(words[2]))

In [ ]:
house_prices_RDD = text_RDD.map(mapper_parse_lines)

In [ ]:
house_prices_RDD.collect()

## Load inflation

In [ ]:
inflation_text_RDD = sc.textFile("/data/inflation.txt")

In [ ]:
def mapper_parse__inflation_lines(line):
    """Parse line into (neighborhoood, inflation) pair"""
    words = line.split()
    return (words[0], float(words[1]))

In [ ]:
inflation_RDD = inflation_text_RDD.map(mapper_parse__inflation_lines)

In [ ]:
inflation_RDD.collect()

## join

In [ ]:
house_prices_RDD.join(inflation_RDD).collect()

In [ ]:
def mapper_multiply_price_inflation(pair):
    inflation_ratio = 1 + pair[1][1]/100.
    return (pair[0], pair[1][0]*inflation_ratio)

In [ ]:
house_prices_nextyear_RDD = house_prices_RDD.join(
    inflation_RDD).map(mapper_multiply_price_inflation)

In [ ]:
house_prices_nextyear_RDD.collect()

## reduce

In [ ]:
def reducer_sum(a,b):
    return a+b

In [ ]:
total_nextyear = house_prices_nextyear_RDD.reduceByKey(reducer_sum)

In [ ]:
total_nextyear.collect()

## Exercise

List neighborhood and house price only for the neighborhoods where inflation is low (less than 4%)

(Advanced: for each of those neighborhoods, find the more expensive house)

## Print DAG

In [ ]:
print(total_nextyear.toDebugString().decode("utf-8"))